##### !!! NOTICE: input1為初版沒有考慮company embedding !!! #####

In [3]:
import numpy as np
import pandas as pd
import os
import json
import re
import monpa
from monpa import utils

/usr/local/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


+---------------------------------------------------------------------+
  Welcome to MONPA: Multi-Objective NER POS Annotator for Chinese
+---------------------------------------------------------------------+
已找到 model檔。Found model file.


In [40]:
monpa.load_userdict("./input/stockName_abbr.txt")

def clearSentence(sentence):
    return re.sub(r'[^\u4e00-\u9fa5]+', '', sentence)

def splitSentence(content, if_clear=1):
    sentence_list = utils.short_sentence(content)
    tokenStr = []
    for sentence in sentence_list:
        if if_clear == 1:
            sentence = clearSentence(sentence)
        tokens = monpa.cut(sentence)
        tokenStr += tokens
    return tokenStr

In [2]:
os.chdir('/Users/arthur/Desktop/factor_investment')

# TransNews

In [8]:
with open('./input2/stockName_abbr.txt', 'r') as infile:
    stockName = infile.read()

In [9]:
stockNameAbbr = []
for i in stockName.split('\n'):
    name = i.split(' ')
    stockNameAbbr.append(name[0])

stockNameAbbr = stockNameAbbr[0:len(stockNameAbbr)-1]

In [10]:
def findCom(content):
    com = []
    for name in stockNameAbbr:
        if name in content:
            com.append(name)
    return com

def del_slash(x):
    return x[0:4]+x[5:7]+x[8:10]

In [11]:
transformedNews = pd.DataFrame()

for i in range(2013, 2022):
    for j in range(1, 13):
        transformedNews = pd.DataFrame()
        segNews = pd.read_csv('./input2/EB_SegAndRawNews/' + str(i) + str(j) + '_cnyesnews.csv')
        segNews = segNews[['date', 'all', 'segmented']]
        segNews = segNews.fillna('')
        for k in segNews.index:
            com = findCom(segNews.loc[k, 'all'])
            for l in com:
                tf = pd.DataFrame({'date': [del_slash(segNews.loc[k, 'date'])], 'all': [segNews.loc[k, 'all']], 'segmented': [segNews.loc[k, 'segmented']], 'com': [l]})
                transformedNews = pd.concat([transformedNews, tf])
            
            if k == segNews.index[-1]:
                transformedNews = pd.DataFrame(transformedNews)
                transformedNews.to_csv('./input2/WB_TransNews/' + str(i) + str(j) + '_cnyesnews.csv', index=False)

# 補救斷詞時可能沒分辨出原有公司簡稱（後來沒用了）

In [30]:
wrongSeg = pd.DataFrame(columns=['file', 'line'])

In [31]:
for loc in ['./input/transformedNews_forEventEmbed/train/', './input/transformedNews_forEventEmbed/valid/', './input/transformedNews_forEventEmbed/test/']:

    input_files = [f for f in os.listdir(loc)  if f.endswith('cnyesnews.csv')]
    
    for file in input_files:
        news = pd.read_csv(loc+file)
        print(file)
        
        for i in news.index:
            if news.loc[i, 'com'] in news.loc[i, 'segmented']:
                continue
            else:
                wrongSeg = pd.concat([wrongSeg, pd.DataFrame( data={'file':[file], 'line':[i]} )])
                news.loc[i, 'segmented'] = splitSentence(news.loc[i, 'all'])
                if news.loc[i, 'com'] in news.loc[i, 'segmented']:
                    print('Fix it!')
                    news.to_csv(loc+file)
                else:
                    print('Still not working.')
            


20186_cnyesnews.csv


/opt/anaconda3/lib/python3.9/site-packages/monpa/crf_layer.py:374: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at  /Users/distiller/project/pytorch/aten/src/ATen/native/TensorCompare.cpp:333.)
  score = torch.where(mask[i].unsqueeze(1), next_score, score)


ValueError: Must have equal len keys and value when setting with an iterable

In [34]:
str(splitSentence(news.loc[i, 'all']))

"['中華', '電', '進軍', '純網銀', '有譜', '謝繼茂', '很多', '銀行', '都', '有', '興趣', '合作', '中華', '電', '今日', '召開', '股東會', '仍', '未', '脫', '以往', '馬拉松', '慣例', '員工', '股東', '輪番', '上場', '暢所欲言', '董事長', '鄭優', '強調', '是', '要', '來', '一起', '打拚', '不', '是', '來', '納涼', '的', '總經理', '謝繼茂', '在', '傍晚', '股東會', '後', '證實', '已', '評估', '確認', '純網銀', '的確', '可', '行', '有助', '在', '金融', '科技', '及', '行動', '支付', '的', '布局', '目前', '很多', '銀行', '都', '有', '合作', '興趣', '中華', '電股東會', '從', '早上', '時', '一路', '開到', '傍晚', '時分', '總共', '開', '了', '個', '小', '時分', '雖', '比', '去年', '多', '了', '約', '半', '小時', '的', '開會', '時間', '但', '也', '算是', '近年', '來', '開會', '時間', '較', '短', '的', '一', '次', '了', '董事長', '鄭優', '針對', '吃到', '飽', '方案', '指出', '當時', '是', '因', '擔心', '市占率', '不斷', '流失', '所以', '提出', '促案', '來', '因應', '鄭優', '表示', '促案', '受到', '民眾', '那麼', '大', '的', '歡迎', '是', '始料未及', '的', '事', '整', '個', '過程', '聽到', '很多', '批評', '也', '聽到', '肯定', '兩', '種', '聲音', '都', '有', '他', '強調', '因為', '支持', '這', '個', '方案', '即使', '有', '壓力', '也', '會', '負責', '到底', '以後', '如果', '還', '要', '推出', '

In [35]:
news.loc[i, 'segmented']

"['中華電', '進軍', '純網銀', '有譜', '謝繼茂', '很多', '銀行', '都', '有', '興趣', '合作', '中華電', '今日', '召開', '股東會', '仍', '未', '脫', '以往', '馬拉松', '慣例', '員工', '股東', '輪番', '上場', '暢', '所', '欲言', '董事長', '鄭優', '強調', '是', '要', '來', '一起', '打拚', '不', '是', '來', '納涼', '的', '總經理', '謝繼茂', '在', '傍晚', '股東會', '後', '證實', '已', '評估', '確認', '純網銀', '的確', '可行', '有助', '在', '金融', '科技', '及', '行動', '支付', '的', '布局', '目前', '很多', '銀行', '都', '有', '合作', '興趣', '中華電', '股東會', '從', '早上', '時', '一路', '開到', '傍晚', '時分', '總共', '開', '了', '個', '小', '時分', '雖', '比', '去年', '多', '了', '約', '半', '小時', '的', '開會', '時間', '但', '也', '算是', '近年', '來', '開會', '時間', '較', '短', '的', '一', '次', '了', '董事長', '鄭優', '針對', '吃到', '飽', '方案', '指出', '當時', '是', '因', '擔心', '市占率', '不斷', '流失', '所以', '提出', '促案', '來', '因應', '鄭優', '表示', '促案', '受到', '民眾', '那麼', '大', '的', '歡迎', '是', '始料未及', '的', '事', '整', '個', '過程', '聽到', '很多', '批評', '也', '聽到', '肯定', '兩', '種', '聲音', '都', '有', '他', '強調', '因為', '支持', '這', '個', '方案', '即使', '有', '壓力', '也', '會', '負責', '到底', '以後', '如果', '還', '要', '推出', '促案',

In [11]:
news = pd.read_csv(loc+file)
for i in news.index:
    if news.loc[i, 'com'] in news.loc[i, 'segmented']:
        continue
    else:
        print(news.loc[i, 'com'])

新產
第一銅
AES
全台
建大
新產
聯合再生
台開
台驊投控
台新金
國聯
新產
聯合再生
光洋科
第一銅
全台
為升
上緯
大成
新產
國統
台灣大
全台
全國
全台
台勝科
日月光投控
合晶
環球晶
大成
第一銅
日月光
台驊投控
元大金
台新金
大立光
日月光投控
同欣電
台塑化
是方
力成
力達
鴻準
金像電
日月光投控
元晶
全新
全家
ABC
華新
台驊投控
元大金
新光金
世紀鋼
新產
新產
全台
及成
台新金
全台
大台中
中華
新產
晶豪科
全新
全台
亞聚
三商
上緯
晶心科
森崴能源
新產
全新
全台
大成
健和興
力旺
創惟
同欣電
M31
威鋒電子
全台
新產
台聚
台光電
群創
AES
亞聚
中鴻
台驊投控
日月光投控
亞聚
聯成
寶齡富錦
日月光
台驊投控
正達
榮創
日月光投控
亞諾法
天鈺
高端疫苗
寶齡富錦
亞諾法
欣大健康
高端疫苗
台康生技
力達
高端疫苗
中鴻
力積
力積電
業強
開發金
國聯
四維航
第一金
大地
全台
全國
為升
高端疫苗
全台
全家
奇力新
日月光投控
明基材
全台
力成
台肥
中華
長榮
萬海
太極
太極
聯合再生
大成
中華
健和興
力旺
創惟
同欣電
M31
威鋒電子
晶豪科
新唐
鈺創
M31
大成
全台
全新
元大高股息
全台
台聚
永豐餘
台聚
亞聚
力成
力成
力成
至上
台新金
第一金
至上
味全
興農
建大
新產
聯合再生
上緯投控
榮創
全台
新產
全國
新產
元晶
元大高股息
中電
台新金
日月光投控
達和
宏捷科
力成
偉詮電
揚明光
美琪瑪
中天
大成
中廣
至上
國聯
譜瑞
凌陽創新
創惟
達和
昇佳電子
威鋒電子
致新
玉晶光
聯合再生
日月光投控
今國光
大成
中華
新產
健和興
力旺
創惟
同欣電
M31
威鋒電子
玉晶光
聯合再生
譜瑞
今國光
佳凌
今國光
台驊投控
全台
致新
中鋼
海德威
凌陽創新
創惟
國票金
聯合再生
晶心科
M31
至上
至上
國聯
中工
TPK
新產
力旺
晶心科
M31
日月光投控
南紡
聯合再生
榮剛
綠界科技
新產
全家
GIS
富邦金
和旺
鮮活果汁
新產
尼得科超眾
元晶
上品
中電
國聯
力旺
晶心科
M31
日月光
中電
高端疫苗
台積電
日月光投控
全台
大成
中華
新產
健和興
創惟
M31
威鋒電子
長榮
世紀鋼
